<img src="images/splash3.png">
Requests data for the past hour and inserts this into SQL. To be ran hourly.


<p><b>What you were doing:</b></p>
<p>&nbsp; &nbsp; only write new tweets </p>

## Setup

In [1]:
import sqlite3 as lite
import sys
import requests
import pandas as pd
import numpy as np
import datetime
import json
import tweepy

In [2]:
# Twitter stuff
CONSUMER_KEY = 'hiZmnSOV63GdqWPqW5MN533T1'# TWITTER_CONSUMER_TOKEN
CONSUMER_SECRET = 'GJa7snhSw8jDQxTVhvoHJyIyJZBjLQikAeuNL0Y3o0Aw61zuOr' # TWITTER_CONSUMER_SECRET
ACCESS_TOKEN = '933287551212904448-uWflGB32dNF8rbgmBxJk1ql5AqemO14' # TWITTER_ACCESS_TOKEN
ACCESS_SECRET = 'boUb7syKFxrCmaJN3T3SHsWu6rDQQfdPxertNIEnXR6X9' # TWITTER_ACCESS_SECRET

## Class & function definitions


In [3]:
class Coin:
    def __init__(self, symbol, comparison_symbol, limit, aggregate):
        self.symbol = symbol
        self.comparison_symbol = comparison_symbol
        self.limit = limit
        self.aggregate = aggregate
        
    def fetch_marketdata(self):    
        url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
                .format(self.symbol.upper(), self.comparison_symbol.upper(), self.limit, self.aggregate)       
        page = requests.get(url)
        data = page.json()['Data']
        df = pd.DataFrame(data)
        df['mid'] = df[["high", "low"]].mean(axis=1)
        df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
        return df
    
    def fetch_tweets(self):
        # Create extractor object
        extractor = twitter_setup()
        # Create a tweet list
        tweets = extractor.search(q ="${}".format(coin_instances[coin].symbol), count=20, tweet_mode="extended", lang="en")
        
        def fulltext(tweet):
            try:
                text = tweet.retweeted_status.full_text
                return(text)
            except:
                text = tweet.full_text
                return(text)

        df = pd.DataFrame(data=[fulltext(tweet) for tweet in tweets], columns=['Tweets'])

        df['Date'] = np.array([tweet.created_at for tweet in tweets])
        df['ID']   = np.array([tweet.id for tweet in tweets])
        df['Len']  = np.array([len(tweet.full_text) for tweet in tweets])
        df['Reply_To_Status'] = np.array([tweet.in_reply_to_user_id for tweet in tweets])
        df['Reply_To_Name'] = np.array([tweet.in_reply_to_screen_name for tweet in tweets])
        df['Coordinates'] = np.array([tweet.coordinates for tweet in tweets])
        df['Retweet_Count'] = np.array([tweet.retweet_count for tweet in tweets])
        df['User_Verified'] = np.array([tweet.user.verified for tweet in tweets])
        df['User_Location'] = np.array([tweet.user.location for tweet in tweets])
        df['User_Followers_Count'] = np.array([tweet.user.followers_count for tweet in tweets])
        df['User_Favorite_Count'] = np.array([tweet.user.favourites_count for tweet in tweets])
        df['Source'] = np.array([tweet.source for tweet in tweets])
        df['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
        df['Favorite_Count'] = np.array([tweet.favorite_count for tweet in tweets])
        df['RTs']    = np.array([tweet.retweet_count for tweet in tweets])        
        return df
        
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication
    api = tweepy.API(auth)
    return api    
    
    
def market_to_db(market_data):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB

    
    with con: #Dump to SQL
        cur = con.cursor()
        cur.execute("SELECT time FROM {} ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol))
        rows = cur.fetchall()
        latest_db_entry = list(rows[0])
        latest_db_entry = int(latest_db_entry[0])
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coin_instances[coin].symbol, con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coin_instances[coin].symbol) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])
            
def tweets_to_db(df_tweets):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB
    
    with con: #Dump to SQL
        cur = con.cursor()
        rows = cur.fetchall()
        print("Going to write tweets to {}_tweets".format(coin_instances[coin].symbol))
        df_tweets.to_sql(name="{}_tweets".format(coin_instances[coin].symbol), con=con, if_exists='append', index=True)

## Fetch data & store in db

In [4]:
fh = open("/hdd/db/tracked_coins.json", 'r')
coinlist = json.load(fh)
coin_instances = [Coin(coinlist[coin], 'BTC','1','1') for coin in range(len(coinlist))] # Make objects. Take all coins in coinlist, convert to BTC, poll 1 entry and aggregate per 1-hour period.

#Get data
#for coin in range(len(coin_instances)):
for coin in range(5):
    try:
        market_data = coin_instances[coin].fetch_marketdata() # Get coindata
        df_tweets = coin_instances[coin].fetch_tweets() # Get tweets
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        market_to_db(market_data)
        tweets_to_db(df_tweets)
    except Exception as e:
        print("Skipping coin", coin_instances[coin], ", Error: ")
        print(e)        

Going to write tweets to KCS_tweets
Going to write tweets to ETH_tweets
Going to write tweets to SC_tweets
Going to write tweets to DGB_tweets
Going to write tweets to REP_tweets


In [5]:
df_tweets = coin_instances[80].fetch_tweets() # Get tweets

In [6]:
df_tweets.tail()

,Tweets,Date,ID,Len,Reply_To_Status,Reply_To_Name,Coordinates,Retweet_Count,User_Verified,User_Location,User_Followers_Count,User_Favorite_Count,Source,Likes,Favorite_Count,RTs
15,Binance is TEMPORARILY accepting new users! HU...,2018-06-14 20:29:14,1007359330876821504,224,None,None,None,0,False,,41,0,442351442,0,0,0
16,Trade crypto on Binance and get 50% discount o...,2018-06-14 20:27:28,1007358882358972416,290,None,None,None,0,False,,1,0,dsfg5434185,0,0,0
17,OmiseGO $OMG price: $9.51 \n\nKucoin distribut...,2018-06-14 20:25:24,1007358365649924096,271,None,None,None,0,False,,10,0,dfg73475818d9fg,0,0,0
18,Binance is TEMPORARILY accepting new users! HU...,2018-06-14 20:22:44,1007357691495419905,229,None,None,None,0,False,,35,0,64334567,0,0,0
19,💡 BINANCE ALERT 💡\n\nBinance registrations are...,2018-06-14 20:14:53,1007355716989063168,231,None,None,None,0,False,,44,2,37544214,0,0,0


In [7]:
df_tweets['Tweets'].iloc[4]

'Price of Decentraland $MANA is: $0.101049 \n\nQUICK! Sign up before Binance closes registration! 💰 🙌  \n\n➡️ https://t.co/S1ruhsLsl3\n\n$XLR $GBG $VIU $ZEN $KMD $PST $MDA $DPY $REP $FCT $MOD $TNT $BITB $NXC $YOYOW $ARGS $PRL $CRB $BCC $XPM $CSNO $XBY $NBT https://t.co/gi51ZUpHSE'